In [ ]:
#!pip uninstall tensorflow -y
#!pip uninstall transformers -y
#!pip install tensorflow==2.3.1
#!pip install transformers

In [ ]:
import tensorflow as tf
from tensorflow import keras
import transformers
from transformers import GPT2Config
from transformers import TFGPT2LMHeadModel
from transformers import GPT2Tokenizer
import os

In [2]:
#!pip install tensorflow_model_optimization
#import tensorflow_model_optimization as tfmot

In [ ]:
tf.config.optimizer.set_jit(True)

In [3]:
dataset_final_path = './dataset_final'
tokenized_data_path = './tokenized_data'
checkpoint_dir = "./ckpt3"

# Prepare a directory to store all the checkpoints.
if not os.path.exists(checkpoint_dir):
    os.makedirs(checkpoint_dir)

In [9]:
element_specs = (tf.TensorSpec(shape=(99,), dtype=tf.int32, name=None),
 tf.TensorSpec(shape=(99,), dtype=tf.int32, name=None))
#Solo seleccionamos 10000 para proba
dataset = tf.data.experimental.load(dataset_final_path,element_specs)#.take(10000)
print("Dataset:\n",len(dataset), dataset.element_spec)


BATCH_SIZE = 8*4*2
BUFFER_SIZE = 10000
shuffled_dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print("Shuffled Dataset:\n",len(shuffled_dataset), shuffled_dataset)
shuffled_dataset = shuffled_dataset.cache()
shuffled_dataset = shuffled_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

tokenizer = GPT2Tokenizer.from_pretrained(tokenized_data_path)
tokenizer.add_special_tokens({
  "eos_token": "</s>",
  "bos_token": "<s>",
  "unk_token": "<unk>",
  "pad_token": "<pad>",
  "mask_token": "<mask>"
})# creating the configurations from which the model can be made
print('Tokenizer Vocab Size:', tokenizer.vocab_size)


Dataset:
 10872699 (TensorSpec(shape=(99,), dtype=tf.int32, name=None), TensorSpec(shape=(99,), dtype=tf.int32, name=None))
Shuffled Dataset:
 169885 <BatchDataset shapes: ((64, 99), (64, 99)), types: (tf.int32, tf.int32)>
Tokenizer Vocab Size: 50000


In [10]:
def get_compiled_model():
    print("Compilando modelo\n=============")
    config = GPT2Config(
        vocab_size = tokenizer.vocab_size,
        n_positions = 512,
        n_ctx = 512, 
        bos_token_id=tokenizer.bos_token_id,
        eos_token_id=tokenizer.eos_token_id,
        pad_token_id = tokenizer.pad_token_id,
        n_head=16 #n_head==Batch Size / GPUs
    )# creating the model
    
    model = TFGPT2LMHeadModel(config)
    # defining our optimizer
    optimizer = tf.keras.optimizers.Adam(learning_rate=3e-5, epsilon=1e-08, 
                                         #clipnorm=1.0 
                                        )
    # definining our loss function
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # defining our metric which we want to observe
    metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')

    # compiling the model
    model.compile(optimizer=optimizer, loss=[loss, *[None] * model.config.n_layer], metrics=[metric])
    
    return model

In [11]:
!rm -rf logs_3

In [12]:
def make_or_restore_model():
    latest = tf.train.latest_checkpoint(checkpoint_dir)
    print('Último Checkpoint:', latest)
    model = get_compiled_model()
    
    if latest:
        print('Restaurando desde :', latest)
        model.load_weights(latest)
    
    return model


def run_training(epochs=1):
    strategy = tf.distribute.MirroredStrategy()

    callbacks = [
    # This callback saves a SavedModel every epoch
    # We include the current epoch in the folder name.
    keras.callbacks.ModelCheckpoint(
        verbose=1,
        filepath=checkpoint_dir + "/ckpt-{epoch}", save_freq="epoch",
        save_weights_only=True
    ),
    tf.keras.callbacks.TensorBoard(log_dir="./logs_3", write_graph=True, update_freq=1000, profile_batch=1000, embeddings_freq=1000)]
    # Open a strategy scope and create/restore the model
    with strategy.scope():
        model = make_or_restore_model()
        
    model.fit(shuffled_dataset, epochs=epochs, callbacks=callbacks, verbose=1)
        
    return model

    
    
bad_words_ids =[ 
    tokenizer.encode('==', add_prefix_space=True), 
    tokenizer.encode('===', add_prefix_space=True),  
    tokenizer.encode('====', add_prefix_space=True),
    tokenizer.encode('<pad>', add_prefix_space=True)]


def test_model(model,text):
    input_ids = tokenizer.encode(text, return_tensors='tf')# getting out output
    print("generando beams para input: ", text,"\n====================================")
    beam_output = model.generate(
      input_ids,
      max_length = 100,
      num_beams = 20,
      bad_words_ids = bad_words_ids,
      temperature = 1,
      #length_penalty=1,
      no_repeat_ngram_size=2,
      repetition_penalty=2,
      num_return_sequences=5
    )
    
    decoded_beams = []
    
    for beam in beam_output:   
        decoded_beams.append(tokenizer.decode(beam))
        print ("\n\n",tokenizer.decode(beam))
        
    return decoded_beams
    

# Epoch 1-5

In [ ]:
history = run_training(epochs=5)

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3')
Último Checkpoint: None
Compilando modelo
Epoch 1/5
INFO:tensorflow:batch_all_reduce: 147 all-reduces with algorithm = nccl, num_packs = 1
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:GPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3').
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/ta

In [ ]:
history.count_params()/1000000

123.849216

In [ ]:
history.config

GPT2Config {
  "activation_function": "gelu_new",
  "attn_pdrop": 0.1,
  "bos_token_id": 0,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 512,
  "n_embd": 768,
  "n_head": 16,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 512,
  "pad_token_id": 1,
  "resid_pdrop": 0.1,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "vocab_size": 50000
}

In [ ]:
bad_words_ids =[ 
    tokenizer.encode('==', add_prefix_space=True), 
    tokenizer.encode('===', add_prefix_space=True),  
    tokenizer.encode('====', add_prefix_space=True),
    tokenizer.encode('<pad>', add_prefix_space=True)]


def test_model(model,text):
    input_ids = tokenizer.encode(text, return_tensors='tf')# getting out output
    print("generando beams para input: ", text,"\n====================================")
    beam_output = model.generate(
      input_ids,
      max_length = 100,
      num_beams = 20,
      bad_words_ids = bad_words_ids,
      temperature = 1,
      #length_penalty=1,
      no_repeat_ngram_size=2,
      repetition_penalty=2,
      num_return_sequences=5
    )
    
    decoded_beams = []
    
    for beam in beam_output:   
        decoded_beams.append(tokenizer.decode(beam))
        print ("\n\n",tokenizer.decode(beam))
        
    return decoded_beams

In [ ]:
test_model(history, "El 27 de enero del año 2010")

generando beams para input:  El 27 de enero del año 2010 


 El 27 de enero del año 2010, se anunció que la serie había sido renovada para una segunda temporada. La primera parte fue estrenada el 13 julio 2011 en Estados Unidos por ABC Family como un spin-off titulado ''NCIS: Naval Criminal Investigative Service''. El 16 agosto 2012 salió al aire su tercera última película titulada "The Big Bang Theory", basada libremente sobre los acontecimientos ocurridos durante las Guerras Clon antes después de Crisis on Infinite Earths (Crisis Final)). En esta nueva versión


 El 27 de enero del año 2010, se anunció que la serie había sido renovada para una segunda temporada. La primera parte fue estrenada el 13 julio 2011 en Estados Unidos por ABC Family como un spin-off titulado ''NCIS: Naval Criminal Investigative Service''. El 11 septiembre 2012 salió al aire su tercera última película titulada "The Big Bang Theory", basada libremente sobre los acontecimientos ocurridos durante las Guerras Clo

['El 27 de enero del año 2010, se anunció que la serie había sido renovada para una segunda temporada. La primera parte fue estrenada el 13 julio 2011 en Estados Unidos por ABC Family como un spin-off titulado \'\'NCIS: Naval Criminal Investigative Service\'\'. El 16 agosto 2012 salió al aire su tercera última película titulada "The Big Bang Theory", basada libremente sobre los acontecimientos ocurridos durante las Guerras Clon antes después de Crisis on Infinite Earths (Crisis Final)). En esta nueva versión',
 'El 27 de enero del año 2010, se anunció que la serie había sido renovada para una segunda temporada. La primera parte fue estrenada el 13 julio 2011 en Estados Unidos por ABC Family como un spin-off titulado \'\'NCIS: Naval Criminal Investigative Service\'\'. El 11 septiembre 2012 salió al aire su tercera última película titulada "The Big Bang Theory", basada libremente sobre los acontecimientos ocurridos durante las Guerras Clon antes después de Crisis on Infinite Earths (Cris

### Guardando el modelo

In [ ]:
from transformers import WEIGHTS_NAME, CONFIG_NAME

import os

output_dir = './model_es_8heads_256pos/'


# creating directory if it is not present
if not os.path.exists(output_dir):
    os.mkdir(output_dir)

model_to_save = history.module if hasattr(history, 'module') else history

output_model_file = os.path.join(output_dir, WEIGHTS_NAME)
output_config_file = os.path.join(output_dir, CONFIG_NAME)


# save model and model configs
history.save_pretrained(output_dir)
model_to_save.config.to_json_file(output_config_file)

# save tokenizer
tokenizer.save_pretrained(output_dir)